In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime as dt
from streamlit_keplergl import keplergl_static

In [2]:
path = '/Users/vickyczada/Library/CloudStorage/OneDrive-Personal/Documents/Career Foundry/Data Visualisation with Python/Achievement 2/NewYork_Bikes_data.csv'

In [3]:
# Obtaining citibikes dataset

df_merged = pd.read_csv(path, index_col = 0)

/var/folders/bm/wxz2f90s7x104nl7tfs2lwdw0000gn/T/ipykernel_1613/1877949233.py:3: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged = pd.read_csv(path, index_col = 0)


In [4]:
df_merged.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'avgTemp', '_merge'],
      dtype='object')

In [5]:
df_merged.drop(columns = {'_merge'}, inplace = True)

In [6]:
df_merged.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
date                   object
avgTemp               float64
dtype: object

In [7]:
df_merged['date'] = pd.to_datetime(df_merged['date'])

In [9]:
# Groupby the count of the trips 
df_group = pd.DataFrame(df_merged.groupby(['date'])['ride_id'].count()).reset_index()

In [10]:
# Renaming the ride_id column as trip_count for clarity.
df_group.rename(columns = {'ride_id':'trip_count'}, inplace = True)
# Merging trip count data with weather data.
df_final = df_merged.merge(df_group, on='date', how='outer', indicator='merge_flag')
print(df_final['merge_flag'].value_counts(dropna = False))
print("Shape of January 1st is", df_final[df_final['date'] == '2022-01-01'].shape) # Check
print("Shape of January 2nd is", df_final[df_final['date'] == '2022-01-02'].shape) # Second check

merge_flag
both          29838806
left_only            0
right_only           0
Name: count, dtype: int64
Shape of January 1st is (20428, 17)
Shape of January 2nd is (43009, 17)


In [11]:
print(df_final.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'avgTemp', 'trip_count', 'merge_flag'],
      dtype='object')


In [12]:
df_merged = df_final.set_index('date')
print(df_merged.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'avgTemp', 'trip_count', 'merge_flag'],
      dtype='object')


In [16]:
df_merged.reset_index(inplace=True)

In [17]:
df_merged.columns

Index(['date', 'ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'avgTemp', 'trip_count', 'merge_flag'],
      dtype='object')

In [18]:
df_merged.shape

(29838806, 17)

In [19]:
df_merged.head()

,date,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,avgTemp,trip_count,merge_flag
0,2021-01-30,34BD74D84D320CF5,electric_bike,2021-01-30 17:30:45.544,2022-05-23 06:17:27.131,W 144 St & Adam Clayton Powell Blvd,7932.1,Frederick Douglass Blvd & W 112 St,7631.23,40.820877,-73.939249,40.801694,-73.957145,casual,NaN,1,both
1,2021-02-15,FA436228F4CD7647,classic_bike,2021-02-15 14:11:20.946,2022-08-10 15:01:30.839,Kent Ave & S 11 St,5062.01,Union Ave & Jackson St,5300.06,40.707645,-73.968415,40.716075,-73.952029,casual,NaN,1,both
2,2021-03-11,8FC77EAE2C0561E4,classic_bike,2021-03-11 22:08:54.461,2022-06-16 09:12:21.565,Union St & 4 Ave,4175.15,Flatbush Ave & Ocean Ave,3704.04,40.677274,-73.982820,40.663657,-73.963014,casual,NaN,1,both
3,2021-03-14,BF45D1E037463AA8,classic_bike,2021-03-14 18:17:59.923,2022-02-02 23:38:01.993,N 12 St & Bedford Ave,5450.04,S Portland Ave & Hanson Pl,4354.05,40.720798,-73.954847,40.685396,-73.974315,member,NaN,1,both
4,2021-03-31,763D82F8CFA1D9D1,classic_bike,2021-03-31 15:35:20.070,2022-09-25 14:33:21.607,Willoughby St & Fleet St,4628.05,DeKalb Ave & Franklin Ave,4528.01,40.691966,-73.981302,40.690648,-73.957462,casual,NaN,1,both


### Creating month and season columns

In [21]:
# Create a month column 

df_merged['date'] = pd.to_datetime(df_merged['date'], format = '%Y-%m-%d')
df_merged['month'] = df_merged['date'].dt.month
df_merged['month'] = df_merged['month'].astype('int')

In [22]:
# Create the season column

df_merged['season'] = [
"winter" if (month == 12 or 1 <= month <= 4)
    else "spring" if (4 < month <= 5)
    else "summer" if (6 <= month <= 9)
    else "fall"
for month in df_merged['month']
    ]

In [23]:
df_merged.shape

(29838806, 19)

In [24]:
df_merged.columns

Index(['date', 'ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'avgTemp', 'trip_count', 'merge_flag', 'month',
       'season'],
      dtype='object')

### Creating the plotly charts

In [25]:
## Groupby

df_merged['value'] = 1 
df_groupby_bar = df_merged.groupby('start_station_name', as_index=False).agg({'value': 'sum'})
top20 = df_groupby_bar.nlargest(20, 'value')

I grouped the data by 'start_station_name' and used the sum function to aggregate the total occurrences for each station. Then, I identified the top 20 stations with the highest counts by applying the .nlargest() method.

In [ ]:
fig = go.Figure(go.Bar(x = top20['start_station_name'], y = top20['value']))
fig.show()

The plot above was created using a code snippet that initializes a bar chart with the go.Bar() function, where go is an alias for the graph_objects module in Plotly. I designed a straightforward bar chart without color customization to effectively visualize the distribution of values across various stations.

In [ ]:
fig = go.Figure(go.Bar(x = top20['start_station_name'], y = top20['value'], marker={'color': top20['value'],'colorscale': 'Blues'}))
fig.show()

By introducing the marker parameter, I applied the Blues color palette to enhance visual clarity, making it easier to distinguish stations with higher counts. Additionally, I used fig.update_layout() to refine the title and axis labels, and adjust the chart's width, ensuring a more polished and informative presentation.

In [ ]:
## Bar chart

fig.update_layout(
    title = 'Top 20 most popular bike stations in NewYork',
    xaxis_title = 'Start stations',
    yaxis_title ='Sum of trips',
    width = 900, height = 600
)

Since the plot is created with Plotly, it inherently offers interactive features, allowing users to hover over each bar to view detailed information such as station names and counts, enhancing the overall user experience.

In [29]:
# Saving the top 20 stations as a CSV file

top20.to_csv('top20.csv')

### Creating a dual-axis line chart for the aggregated bike trips and temperatures in plotly.

In [ ]:
# Aggregating the data by datetime
df_aggregated = df_merged.groupby('date').agg({
    'trip_count': 'mean',  # Using 'mean' for daily bike rides
    'avgTemp': 'mean'  # Using 'mean' for daily temperature
}).reset_index()

# Creating subplot with two y-axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Adding trace for daily bike rides
fig.add_trace(
        go.Scatter(x=df_aggregated['date'], y=df_aggregated['trip_count'], name='Daily bike rides', line=dict(color='blue')),
    secondary_y=False,
)

# Adding trace for daily temperature
fig.add_trace(
    go.Scatter(x=df_aggregated['date'], y=df_aggregated['avgTemp'], name='Daily temperature', line=dict(color='red')),
    secondary_y=True,
)

In [ ]:
# using fig.update_layout() to add a title and adjust the size (customizing the chart)
fig.update_layout(
        title = 'Daily bike rides and Temperature in 2022 NewYork',
        xaxis_title = 'Date',
        yaxis = dict(
            title = 'Daily Total Bike Rides',
            titlefont = dict(color = 'blue'),
            tickfont = dict(color = 'blue'),
            tickformat=".1s"
        ),
        yaxis2 = dict(
            title = 'Daily Avg Temperature',
            titlefont = dict(color = 'red'),
            tickfont = dict(color = 'red'),
            anchor = 'x',
            overlaying = 'y',
            side = 'right',
           tickformat="d"
        ),
        legend = dict(
            x = 0,
            y = 1.1,
            orientation = 'h'
        )
    ) 

In [32]:
df_merged.columns

Index(['date', 'ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'avgTemp', 'trip_count', 'merge_flag', 'month',
       'season', 'value'],
      dtype='object')

In [34]:
df_analysis = df_merged.drop(columns = {'ride_id','started_at', 'ended_at','start_station_id','end_station_id','start_lat', 'start_lng', 'end_lat', 'end_lng','member_casual','merge_flag', 'month'})

In [35]:
df_analysis.columns

Index(['date', 'rideable_type', 'start_station_name', 'end_station_name',
       'avgTemp', 'trip_count', 'season', 'value'],
      dtype='object')

In [36]:
df_analysis.shape

(29838806, 8)

### Creating a random split

In [37]:
np.random.seed(32)
red = np.random.rand(len(df_analysis)) <= 0.92

In [38]:
small = df_analysis[~red]

In [39]:
small.shape

(2385111, 8)

In [40]:
small.to_csv('reduced_data_to_plot_8.csv',index = False)

In [41]:
df_analysis.to_csv('subset_newyork_data.csv')